In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import csv
import os

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield instance

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# ONE RUN
#############################################

agent = simulation.simulator(
                     connection_label = "lonely_worker",
                     q_network_type = 'simple',
                     target_q_network_type = 'simple',
                     gamma = 0.999,
                     target_update_freq = 1000,
                     train_freq = 5,
                     num_burn_in = 300,
                     batch_size = 70,
                     optimizer = optimizers.RMSprop(lr=0.001, rho=0.95),
                     loss_func = "mse",
                     max_ep_length = 4000,
                     experiment_id = "best_param_nominal",
                     model_checkpoint = True,
                     opt_metric = None,
                     
                    # environment parameters
                     net_file = "cross.net.xml",
                     route_file = "cross.rou.xml",
                     demand = "nominal",
                     state_shape = (1,11),
                     num_actions = 2,
                     use_gui = False,
                     delta_time = 10,
                 
                    # memory parameters
                     max_size = 100000,
                     
                    # additional parameters
                 
                     policy = "epsgreedy_decay",
                     eps = 1,
                     num_episodes = 300,
                     monitoring = True)
agent.train()
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")
# agent.evaluate(runs=1, use_gui= False)

In [ ]:
agent.ddqn.train(env = agent.env, num_episodes = 300, policy = agent.policy, connection_label = agent.connection_label)

In [3]:
# GRIDSEARCH
###############################################################

exp_id = "Desperation"

# to search tomorrow: optimiser, delta time, exploration decay, network architecture

param = {
    #"batch_size" : [100], # 30,50,60,[70]
    "target_update_frequency" : [1000, 5000, 10000], # 10000,20000,30000,[40000]
    #"gamma" : [0.99,0.995], # 0.98,0.99,0.995,[0.999]
    #"train_freq" : [3,4], # 2,3,4,[5]
    #"max_size" : [50000,70000], # 20000,50000,70000,[100000]
    #"max_ep_length" : [2000,3000], # 1000,2000,3000,[4000]
    "policy" : ["epsGreedy", "linDecEpsGreedy", "epsgreedy_decay"],
    "eps" : [0.5, 0.3, 0.1],
    "lr" : [0.0001, 0.001, 0.01]
}

param_grid = iter_params(**param)

def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all objects
    
    # print('Run', position + 1, '-- parameters', args)
    
    agent = simulation.simulator(
                    connection_label = position + 1,
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = 0.99,
                    target_update_freq = args[0],
                    train_freq = 4,
                    num_burn_in = 300,
                    batch_size = 40,
                    optimizer = optimizers.RMSprop(lr= args[3], rho=0.95),
                    loss_func = "mse",
                    max_ep_length = 1000,
                    experiment_id = exp_id,
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    net_file = "cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,11),
                    num_actions = 2,
                    use_gui = False,
                    delta_time = 10,

                   # memory parameters
                    max_size = 100000,

                   # additional parameters

                    policy = args[1],
                    eps = args[2],
                    num_episodes = 300,
                    monitoring = True)
    
    # print("training agent", position + 1)
    agent.train()
    # print("evaluating agent", position + 1)
    result = agent.evaluate(runs = 3)

    return (multiprocessing.current_process().name, position + 1, args, result)


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
    with open(os.path.join("./logs",exp_id,"GS_results"), "a",newline='') as file:
        writer = csv.writer(file, dialect = 'excel')
        writer.writerow(["Process", "Run", "Parameters", "Result"])
    
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        with open(os.path.join("./logs",exp_id,"GS_results"), "a",newline='') as file:
            writer = csv.writer(file, dialect = 'excel')
            writer.writerow(results[-1])                
        print('%s -- [RESULTS]: Run %s -- Parameters %s -- Mean duration %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
    scores = [result[-1] for result in results]
    lowest = min(scores)
    winner = results[scores.index(lowest)]
    return winner, results

multiprocessing.freeze_support()
winner, results = gridsearch(param_grid)

Started process # 1
Started process # 2
Started process # 3
Started process # 4
Started process # 5
Started process # 6
Started process # 7
Started process # 8
Run 1 -- running episode 1 / 300
Run 2 -- running episode 1 / 300
Run 7 -- running episode 1 / 300
Run 6 -- running episode 1 / 300
Run 3 -- running episode 1 / 300
Run 8 -- running episode 1 / 300
Run 4 -- running episode 1 / 300
Run 5 -- running episode 1 / 300
Run 2 -- running episode 2 / 300
Run 1 -- running episode 2 / 300
Run 8 -- running episode 2 / 300
Run 5 -- running episode 2 / 300
Run 6 -- running episode 2 / 300
Run 4 -- running episode 2 / 300
Run 7 -- running episode 2 / 300
Run 3 -- running episode 2 / 300
Run 2 -- running episode 3 / 300
Run 1 -- running episode 3 / 300
Run 8 -- running episode 3 / 300
Run 5 -- running episode 3 / 300
Run 4 -- running episode 3 / 300
Run 7 -- running episode 3 / 300
Run 3 -- running episode 3 / 300
Run 6 -- running episode 3 / 300
Run 2 -- running episode 4 / 300
Run 1 -- runnin

Run 3 -- running episode 27 / 300
Run 6 -- running episode 16 / 300
Run 1 -- running episode 34 / 300
Run 7 -- running episode 33 / 300
Run 5 -- running episode 34 / 300
Run 4 -- running episode 34 / 300
Run 2 -- running episode 35 / 300
Run 8 -- running episode 34 / 300
Run 1 -- running episode 35 / 300
Run 7 -- running episode 34 / 300
Run 5 -- running episode 35 / 300
Run 3 -- running episode 28 / 300
Run 4 -- running episode 35 / 300
Run 2 -- running episode 36 / 300
Run 8 -- running episode 35 / 300
Run 1 -- running episode 36 / 300
Run 7 -- running episode 35 / 300
Run 6 -- running episode 17 / 300
Run 5 -- running episode 36 / 300
Run 4 -- running episode 36 / 300
Run 2 -- running episode 37 / 300
Run 3 -- running episode 29 / 300
Run 8 -- running episode 36 / 300
Run 1 -- running episode 37 / 300
Run 7 -- running episode 36 / 300
Run 5 -- running episode 37 / 300
Run 4 -- running episode 37 / 300
Run 2 -- running episode 38 / 300
Run 1 -- running episode 38 / 300
Run 8 -- runni

Run 6 -- running episode 33 / 300
Run 8 -- running episode 66 / 300
Run 2 -- running episode 68 / 300
Run 1 -- running episode 68 / 300
Run 3 -- running episode 50 / 300
Run 7 -- running episode 67 / 300
Run 4 -- running episode 68 / 300
Run 5 -- running episode 68 / 300
Run 6 -- running episode 34 / 300
Run 8 -- running episode 67 / 300
Run 1 -- running episode 69 / 300
Run 2 -- running episode 69 / 300
Run 7 -- running episode 68 / 300
Run 3 -- running episode 51 / 300
Run 4 -- running episode 69 / 300
Run 5 -- running episode 69 / 300
Run 6 -- running episode 35 / 300
Run 1 -- running episode 70 / 300
Run 8 -- running episode 68 / 300
Run 2 -- running episode 70 / 300
Run 7 -- running episode 69 / 300
Run 4 -- running episode 70 / 300
Run 5 -- running episode 70 / 300
Run 3 -- running episode 52 / 300
Run 1 -- running episode 71 / 300
Run 6 -- running episode 36 / 300
Run 8 -- running episode 69 / 300
Run 2 -- running episode 71 / 300
Run 7 -- running episode 70 / 300
Run 4 -- runni

Run 2 -- running episode 100 / 300
Run 1 -- running episode 102 / 300
Run 7 -- running episode 100 / 300
Run 4 -- running episode 101 / 300
Run 8 -- running episode 100 / 300
Run 3 -- running episode 75 / 300
Run 5 -- running episode 100 / 300
Run 2 -- running episode 101 / 300
Run 6 -- running episode 51 / 300
Run 1 -- running episode 103 / 300
Run 7 -- running episode 101 / 300
Run 8 -- running episode 101 / 300
Run 4 -- running episode 102 / 300
Run 5 -- running episode 101 / 300
Run 2 -- running episode 102 / 300
Run 1 -- running episode 104 / 300
Run 7 -- running episode 102 / 300
Run 3 -- running episode 76 / 300
Run 8 -- running episode 102 / 300
Run 5 -- running episode 102 / 300
Run 4 -- running episode 103 / 300
Run 2 -- running episode 103 / 300
Run 6 -- running episode 52 / 300
Run 1 -- running episode 105 / 300
Run 7 -- running episode 103 / 300
Run 8 -- running episode 103 / 300
Run 5 -- running episode 103 / 300
Run 4 -- running episode 104 / 300
Run 3 -- running episode

Run 1 -- running episode 135 / 300
Run 7 -- running episode 132 / 300
Run 8 -- running episode 133 / 300
Run 4 -- running episode 131 / 300
Run 5 -- running episode 133 / 300
Run 2 -- running episode 134 / 300
Run 6 -- running episode 67 / 300
Run 1 -- running episode 136 / 300
Run 7 -- running episode 133 / 300
Run 8 -- running episode 134 / 300
Run 3 -- running episode 100 / 300
Run 4 -- running episode 132 / 300
Run 5 -- running episode 134 / 300
Run 2 -- running episode 135 / 300
Run 1 -- running episode 137 / 300
Run 7 -- running episode 134 / 300
Run 8 -- running episode 135 / 300
Run 3 -- running episode 101 / 300
Run 4 -- running episode 133 / 300
Run 5 -- running episode 135 / 300
Run 2 -- running episode 136 / 300
Run 6 -- running episode 68 / 300
Run 1 -- running episode 138 / 300
Run 8 -- running episode 136 / 300
Run 7 -- running episode 135 / 300
Run 4 -- running episode 134 / 300
Run 5 -- running episode 136 / 300
Run 3 -- running episode 102 / 300
Run 2 -- running episo

...

Run 74 -- running episode 227 / 300
Run 81 -- running episode 17 / 300
Run 77 -- running episode 288 / 300
Run 80 -- running episode 66 / 300
Run 79 -- running episode 110 / 300
Run 78 -- running episode 179 / 300
Run 71 -- running episode 289 / 300
Run 75 -- running episode 290 / 300
Run 77 -- running episode 289 / 300
Run 81 -- running episode 18 / 300
Run 78 -- running episode 180 / 300
Run 74 -- running episode 228 / 300
Run 77 -- running episode 290 / 300
Run 81 -- running episode 19 / 300
Run 80 -- running episode 67 / 300
Run 79 -- running episode 111 / 300
Run 78 -- running episode 181 / 300
Run 71 -- running episode 290 / 300
Run 77 -- running episode 291 / 300
Run 75 -- running episode 291 / 300
Run 81 -- running episode 20 / 300
Run 78 -- running episode 182 / 300
Run 77 -- running episode 292 / 300
Run 74 -- running episode 229 / 300
Run 80 -- running episode 68 / 300
Run 81 -- running episode 21 / 300
Run 79 -- running episode 112 / 300
Run 78 -- running episode 183 / 300


Run 74 -- running episode 263 / 300
Run 81 -- running episode 67 / 300
Run 78 -- running episode 229 / 300
Run 74 -- running episode 264 / 300
Run 80 -- running episode 93 / 300
Run 79 -- running episode 138 / 300
Run 78 -- running episode 230 / 300
Run 81 -- running episode 68 / 300
Run 74 -- running episode 265 / 300
Run 79 -- running episode 139 / 300
Run 78 -- running episode 231 / 300
Run 81 -- running episode 69 / 300
Run 74 -- running episode 266 / 300
Run 80 -- running episode 94 / 300
Run 79 -- running episode 140 / 300
Run 78 -- running episode 232 / 300
Run 74 -- running episode 267 / 300
Run 81 -- running episode 70 / 300
Run 79 -- running episode 141 / 300
Run 80 -- running episode 95 / 300
Run 74 -- running episode 268 / 300
Run 78 -- running episode 233 / 300
Run 81 -- running episode 71 / 300
Run 79 -- running episode 142 / 300
Run 74 -- running episode 269 / 300
Run 78 -- running episode 234 / 300
Run 81 -- running episode 72 / 300
Run 80 -- running episode 96 / 300
Ru

Run 81 -- running episode 117 / 300
Run 78 -- running episode 282 / 300
Run 79 -- running episode 193 / 300
Run 74 -- running episode 299 / 300
Run 81 -- running episode 118 / 300
Run 78 -- running episode 283 / 300
Run 80 -- running episode 122 / 300
Run 79 -- running episode 194 / 300
Run 78 -- running episode 284 / 300
Run 81 -- running episode 119 / 300
Run 79 -- running episode 195 / 300
Run 74 -- running episode 300 / 300
Run 78 -- running episode 285 / 300
Run 81 -- running episode 120 / 300
Run 80 -- running episode 123 / 300
Run 79 -- running episode 196 / 300
Run 78 -- running episode 286 / 300
Run 81 -- running episode 121 / 300
Run 79 -- running episode 197 / 300
Run 78 -- running episode 287 / 300
Run 80 -- running episode 124 / 300
Run 81 -- running episode 122 / 300
Run 79 -- running episode 198 / 300
Run 78 -- running episode 288 / 300
Run 81 -- running episode 123 / 300
Run 79 -- running episode 199 / 300
Run 80 -- running episode 125 / 300
Run 78 -- running episode 28

Run 81 -- running episode 194 / 300
Run 79 -- running episode 278 / 300
Run 80 -- running episode 161 / 300
Run 81 -- running episode 195 / 300
Run 79 -- running episode 279 / 300
Run 81 -- running episode 196 / 300
Run 79 -- running episode 280 / 300
Run 81 -- running episode 197 / 300
Run 79 -- running episode 281 / 300
Run 80 -- running episode 162 / 300
Run 81 -- running episode 198 / 300
Run 79 -- running episode 282 / 300
Run 80 -- running episode 163 / 300
Run 81 -- running episode 199 / 300
Run 79 -- running episode 283 / 300
Run 81 -- running episode 200 / 300
Run 79 -- running episode 284 / 300
Run 80 -- running episode 164 / 300
Run 79 -- running episode 285 / 300
Run 81 -- running episode 201 / 300
Run 79 -- running episode 286 / 300
Run 80 -- running episode 165 / 300
Run 81 -- running episode 202 / 300
Run 79 -- running episode 287 / 300
Run 81 -- running episode 203 / 300
Run 79 -- running episode 288 / 300
Run 80 -- running episode 166 / 300
Run 81 -- running episode 20

Run 80 -- running episode 253 / 300
Run 80 -- running episode 254 / 300
Run 80 -- running episode 255 / 300
Run 80 -- running episode 256 / 300
Run 80 -- running episode 257 / 300
Run 80 -- running episode 258 / 300
Run 80 -- running episode 259 / 300
Run 80 -- running episode 260 / 300
Run 80 -- running episode 261 / 300
Run 80 -- running episode 262 / 300
Run 80 -- running episode 263 / 300
Run 80 -- running episode 264 / 300
Run 80 -- running episode 265 / 300
Run 80 -- running episode 266 / 300
Run 80 -- running episode 267 / 300
Run 80 -- running episode 268 / 300
Run 80 -- running episode 269 / 300
Run 80 -- running episode 270 / 300
Run 80 -- running episode 271 / 300
Run 80 -- running episode 272 / 300
Run 80 -- running episode 273 / 300
Run 80 -- running episode 274 / 300
Run 80 -- running episode 275 / 300
Run 80 -- running episode 276 / 300
Run 80 -- running episode 277 / 300
Run 80 -- running episode 278 / 300
Run 80 -- running episode 279 / 300
Run 80 -- running episode 28

In [ ]:
omega_test = 1-1e-10
omega_test

In [ ]:
decay_test = []
eps_test = 1
for i in range(100000):
    eps_test *= omega_test ** i
    decay_test.append(eps_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(100000),decay_test)

In [7]:
pd.DataFrame(results).sort_values(by = 3)

,0,1,2,3
2,Process-13,5,"(1000, epsGreedy, 0.3, 0.001)",104.302342
18,Process-15,22,"(1000, epsgreedy_decay, 0.3, 0.0001)",105.364934
64,Process-12,64,"(10000, linDecEpsGreedy, 0.5, 0.0001)",108.132788
0,Process-10,2,"(1000, epsGreedy, 0.5, 0.001)",108.581432
3,Process-16,8,"(1000, epsGreedy, 0.1, 0.001)",108.830234
8,Process-9,13,"(1000, linDecEpsGreedy, 0.3, 0.0001)",108.841801
59,Process-10,61,"(10000, epsGreedy, 0.1, 0.0001)",112.823564
19,Process-9,25,"(1000, epsgreedy_decay, 0.1, 0.0001)",121.118484
78,Process-15,79,"(10000, epsgreedy_decay, 0.1, 0.0001)",121.723504
16,Process-9,17,"(1000, linDecEpsGreedy, 0.1, 0.001)",132.069544
